In [32]:
import cvxpy as cp
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


inputnodes = [0,6]
inputedges = [(0,5),(0,3),(3,4),(4,5),(4,6),(5,2),(3,1),(1,6),(6,2)]

non_member = [3,4,5,6]
source = 0
recievers = [1,2]


preG = nx.Graph()
preG.add_nodes_from(inputnodes)
preG.add_edges_from(inputedges)
directE = []
for n, nbrs in preG.adj.items():
    for nbr, eattr in nbrs.items():
        directE.append((n,nbr))


G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(directE)

undirected_edges = set(inputedges)

vertices = np.arange(inputnodes[1]+1)
print(vertices)

edges = set(directE)
print(edges)


fstar_constraints = []
flow_constraints = []
capacity_constraints = []
variable_constraints = []


g = {(i, a): cp.Variable() for a in edges for i in recievers if a[1] == int(i) } # f* == incoming reciever flow

fstar_constraints = [(-1 + sum(g[i, a] for a in edges for i in recievers if a[1] == int(i) )) >= 0]

c = {(i, a): cp.Variable() for a in edges for i in recievers}   # fia <= c(a)

for i in recievers:
    for a in edges:
        variable_constraints += [c[i,a] >= 0]

d = {(i, a): cp.Variable() for a in edges for i in recievers if (a[0] in non_member or a[1] in non_member) }   # outfrom non-member == into non-memebr
e = {(i, a): cp.Variable() for a in edges for i in recievers if a[1] == 0}   # into source
f = {(i, a): cp.Variable() for a in edges for i in recievers if a[0] == int(i)}   # outfrom reciever
g = {(i, a): cp.Variable() for a in edges for i in recievers if a[1] == int(i)}   # into reciever





for i in recievers:
    for a in edges:
        equation = c[i,a]
        if a[1] in non_member:
            equation += d[i,a]
        if a[0] in non_member:
            equation -= d[i,a]
        if a[1] == source:
            equation += e[i,a]
        if a[0] == i:
            equation += f[i,a]
        if a[1] == i:
            equation -= g[i,a]
        
        flow_constraints += [equation >= 0]

b = {a1: cp.Variable() for a1 in undirected_edges}   # fia <= c(a)


for a1 in undirected_edges:
    a2 = (a1[1], a1[0]) 
    equation1 = b[a1]
    equation2 = b[a1]
    for i in recievers:
        equation1 -= c[i, a1]
        equation2 -= c[i, a2]

    
    capacity_constraints += [equation1 >= 0]
    capacity_constraints += [equation2 >= 0]

objective = cp.Minimize(sum(b[a1] for a1 in undirected_edges))
    










# f_star = cp.Variable()  


# orientation_constraints = []
# flow_constraints = []
# equalrate_constraints = []

# c =  {a : cp.Variable() for a in edges}

# for a in edges:
#     orientation_constraints += [c[a] >= 0]      #for every in 2E

# for a1 in undirected_edges:         # for every in E
#     a2 = (a1[1], a1[0]) 
#     orientation_constraints += [ c[a1] + c[a2] == 1]


# fia = {(i, a): cp.Variable() for a in edges for i in recievers}

# for i in recievers: 
#      for a in edges:
#           flow_constraints += [fia[i,a] <= c[a]]
#           flow_constraints += [fia[i,a] >= 0]


# for i in recievers:
#     for j in non_member:
#         incoming_edges = [(u, v) for u, v in edges if v == int(j)] 
#         outcoming_edges = [(u, v) for u, v in edges if u == int(j)] 
#         incoming_flow = sum(fia[i, x] for x in incoming_edges)
#         outcoming_flow = sum(fia[i, y] for y in outcoming_edges)
#         flow_constraints += [incoming_flow == outcoming_flow]
#         print(incoming_edges)

#     incoming_flow_source = sum(fia[i, x] for x in edges if x[1] == source)  
    
#     outcoming_flow_reciever = sum(fia[i, x] for x in edges if x[0] == int(i)) 
#     incoming_flow_reciever = sum(fia[i, x] for x in edges if x[1] == int(i))  

#     flow_constraints += [incoming_flow_source == 0]
#     flow_constraints += [outcoming_flow_reciever == 0]
#     equalrate_constraints += [f_star == incoming_flow_reciever]




# objective = cp.Maximize(f_star)    


constraints = fstar_constraints + flow_constraints + capacity_constraints + variable_constraints


problem = cp.Problem(objective, constraints)


problem.solve()

print("minimal objective:", sum(b[a1] for a1 in undirected_edges).value)



print((-1 + sum(g[i, a] for a in edges for i in recievers if a[1] == int(i) )).value)

[0 1 2 3 4 5 6]
{(6, 2), (3, 4), (4, 3), (3, 1), (6, 1), (5, 4), (0, 3), (4, 6), (6, 4), (3, 0), (4, 5), (5, 0), (2, 6), (0, 5), (1, 6), (2, 5), (1, 3), (5, 2)}
minimal objective: -2.1218277753866558e-10
-1.6638216255740397
